# Python для сбора и анализа данных

## Практикум 6.2. Управление браузером с Selenium: поиск элементов на странице и запросы XPATH

*Алла Тамбовцева, НИУ ВШЭ*

### Установка и подготовка к работе

Библиотека Selenium – библиотека для управления браузером с помощью Python. Она позволяет запускать браузер и имитировать действия пользователя в нём. 

Зачем это может понадобиться? Во-первых, такое «встраивание» в браузер в большинстве случаев позволяет решить проблему с возникающими капчами и иными ограничениями, так как настройки сайта не распознают, что запрос к исходному коду страницы производится автоматически. Во-вторых, необходимость имитации действий в браузере неизбежно возникает при обработке динамических веб-страниц, где некоторые элементы (окна, графики, таблицы) появляются только при определенных действиях пользователя, например, при скроллинге или наведении мышкой. В-третьих, библиотека может быть полезна в случаях, если доступ к API сайта или базы данных получить довольно сложно, но при работе в браузере информация доступна (в таком случае можем залогиниться через Python как пользователь и потихоньку выгружать данные).

Установим библиотеку:

In [ ]:
!pip install selenium

Сначала импортируем библиотеку полностью, проверим, что всё идёт по плану:

In [1]:
import selenium

Теперь импортируем отдельные компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [3]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запустим браузер Chrome средствами Selenium (в модуле есть функции для разных браузеров, но лучше всего работать с Chrome или Firefox):

In [4]:
br = wd.Chrome()

Если вы исполняете код выше первый раз, это может занять некоторое время, так как функция `Chrome()` сначала скачает и установит подходящую версию драйвера для Chrome, который будет обеспечивать связь между Selenium и той версией браузера Chrome, которая установлена у вас на компьютере. В последующие разы всё должно происходить быстрее.

В результате запуска строки выше должно открыться новое окно браузера Chrome. Оно пустое и, скорее всего, с всплывающим предупреждением о том, что браузером управляет автоматизированное тестовое ПО. Это нормально, мы «имитировали» открытие браузера от лица пользователя, теперь, если мы будем в этом окне переходить по ссылкам на сайты, сайты не будут воспринимать запросы как автоматические (в отличие от запросов через модуль `requests`, из-за автоматических или слишком быстрых запросов, собственно, могут возникать те же капчи).

Открывшееся окно браузера в течение работы закрывать нельзя. Python будет там выполнять действия, а мы будем их отслеживать со стороны.
Приступим к работе.

Открываем новое окно на всю ширину экрана на случай, если какие-то элементы в маленьком окне будут мешать или накладываться друг на друга (так бывает с рекламой, всплывающими окнами и подобным):

In [5]:
br.maximize_window()

### Часть 1:  поиск элементов на странице и запросы XPATH

Начнём с простого примера – сайта книжного магазина «Библио-Глобус». Вообще этот сайт можно парсить и без Selenium, он пока не блокирует автоматические запросы, но зато другие сайты магазинов или онлайн-кинотеатров с похожей структурой умеют это делать. 

Отправим запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [6]:
br.get("https://www.biblio-globus.ru/")

В окне должна открыться главная страница сайта. Найдём на странице поле для поиска интересующего товара. Для этого нам понадобится метод `.find_element()`, он применяется к объекту *WebDriver* (у нас `br`). Вообще в Selenium есть два метода для поиска, по аналогии с методами `.find()` и `.find_all()` в BeautifulSoup:

* `.find_element()` – поиск одного элемента, возвращает один результат, если подходящих элементов несколько, возвращается первый;
* `.find_elements()` – поиск нескольких элементов, возвращает список результатов.

Искать будем по id объекта, его можем найти в исходном коде страницы, в инструментах разработчика, или через расширение *SelectorGadget* (установить можно [здесь](https://chromewebstore.google.com/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb?pli=1)).

In [7]:
search = br.find_element(By.ID, "SearchBooks")

В SelectorGadget id объекта отображается с `#`, классы с `.`, в нём же можно запросить более сложный запрос `XPATH`, но об этом поговорим позже. Сейчас в `search` у нас хранится объект Selenium, к которому можно применять различные методы. Так как мы нашли поле для поиска, которое можно заполнять, мы воспользуемся методом `.send_keys()`, который введёт в это поле текст. Запросим книги по Python:

In [8]:
search.send_keys("Python")

В открытом окне должен появиться текст и выпадающее меню с названиями книг. Магия! 

**Примечание.** Если мы нашли объект некорректно, например, не само поле (в HTML обычно с тэгом `<input>`), а рамочку вокруг него или раздел, внутри которого это поле находится, то есть те элементы, которые не подразумевают интерактива в виде ввода значений, в ответ на `.send_keys()` мы получим ошибку `element not interactable`.

Для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

In [9]:
search.send_keys(Keys.ENTER)

Отлично! Теперь нужно поработать с результатами поиска. Предположим, что мы пишем код – основу для какой-нибудь программы или приложения, которая позволяет пользователю по ключевым словам выполнить поиск и отсортировать результаты по какому-нибудь критерию, доступному на сайте. Например, по цене, от меньшей к большей. 

Найдём выпадающее меню с нужными опциями на странице по названию тэга.

In [10]:
options = br.find_element(By.TAG_NAME, "select")

По аналогии с предыдущим примером заполнения ввода поля, выбор опций в обычных выпадающих меню с тэгом `<select>` можно осуществить с помощью того же метода `.send_keys()`.

In [11]:
options.send_keys(" По цене (от меньшей к большей) ")

Отлично! Давайте ещё найдём поле для галочки «В наличии» и кликнем на него!

In [12]:
tick = br.find_element(By.CLASS_NAME, "custom-control-label")
tick.click()

Технически, нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью BeautifulSoup. 

In [13]:
html = br.page_source

Попробуем найти карточки с найденными товарами!

In [14]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(html)

In [16]:
divs = soup.find_all("div", {"class" : "product"})
print(divs[3]) # пример одной карточки

<div class="product">
<a class="img_link" href="/product/10776656"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10776656.jpg&amp;width=200"/></a>
<div class="text">
<div class="product-sm-used-placeholder"></div>
<div class="author">Кольцов Д. М.</div>
<h3><a href="/product/10776656" id="p_title_10776656">Справочник PYTHON.  Кратко, быстро, под рукой</a></h3>
<div class="prices_item">
<div class="price_item_wrapper"><span class="price_item_title">Цена на сайте:</span> <div class="price_item_block"><s>569 ₽</s> <span class="price_item_new price_item_with_discount">512 ₽</span></div></div>
</div>
<p class="status im_status_title">
<span style="color: #3ba155;"> в наличии</span>
<span class="product-qty">3 шт.</span>
</p>
<div class="buttons row">
<div class="col-9 pl-0">
<span class="btn btn-block btn-primary" id="add_10776656" onclick="AddToBasket(10776656)">В корзину</span>
<a class="btn btn-primary add_but" href="/Basket/Detail" id="added_107

Напишем функцию `get_item()`, которая будет принимать на вход одну карточку, то есть один элемент – объект BeautifulSoup из `divs`, а возвращать список характеристик товара:

* название товара;
* ссылка на страницу;
* цена.

In [17]:
def get_item(item):
    name = item.find("h3").text
    link = "https://www.biblio-globus.ru/" + item.find("h3").find("a").get("href")
    price = item.find("div", {"class" : "price_item_block"}).text.replace("\xa0", " ").strip()
    return [name, link, price]

Применим эту функцию ко всем элементам в `divs`:

In [18]:
res = [get_item(d) for d in divs]

Но давайте мыслить более глобально – результаты не помещаются на одной странице, поэтому для общего решения нам надо научиться выяснять, сколько страниц пролистывать для сбора всех товаров по ключевым словам. 

Для этого нам нужно найти кнопку со стрелкой `»»` и внимательно её изучить. Давайте попробуем найти эту стрелку разными способами, чтобы познакомиться с разными атрибутами в `By`.

Сначала найдём её последовательно по классам и тэгам (последовательно, потому что стрелка представлена ссылкой, а ссылок на странице много, нужно сначала зафиксировать более общий раздел, где её искать) и сохраним как `arrow`.

**Подсказка.** Сначала имеет смысл найти список со ссылками на разные страницы по классу.

In [19]:
ul = br.find_element(By.CLASS_NAME, "pagination")

In [20]:
arrow = ul.find_elements(By.TAG_NAME, "a")[-1]

Посмотрим, что внутри. Можем запросить весь код HTML, который есть внутри найденного объекта (его тип *WebElement*):

In [21]:
arrow.get_attribute('innerHTML')

'<span aria-hidden="true">»»</span><span class="sr-only">Next</span>'

В нашем случае полный код не очень интересен, потому что ссылка, которая отправляет нас на последнюю страницу с результатами, хранится не внутри тэгов, а в отдельном атрибуте `href` (как и обычно). Текст `»»` нам, скорее, полезен просто для того, чтобы убедиться, что мы нашли то, что нужно :) 

Запросим аналогичным образом `href`:

In [22]:
arrow.get_attribute('href')

'https://www.biblio-globus.ru/catalog/search?query=Python&page=10&sort=5&instock=on&cat=0&isdiscount='

Вот эта информация уже гораздо полезнее, внутри этой ссылки есть указание на то, какая страница является последней. То есть, нам не придётся кликать на стрелки *Далее* до тех пор, пока мы не дойдём до конца, через какой-нибудь цикл while, мы сможем написать код, который будет пролистывать 10 страниц и забирать результаты. 

Посмотрим на ещё один способ поиска объектов – найдём нужную стрелку проще – по тексту ссылки.

In [23]:
br.find_element(By.PARTIAL_LINK_TEXT, "»»")

<selenium.webdriver.remote.webelement.WebElement (session="4f308cb63f873f9d0ce50893811fd93a", element="B17C842BC9A8E4E3FF0E62DB02EC7F69_element_28")>

Использованные выше способы поиска рабочие, но не всегда надёжные и универсальные. В первом случае смущает поиск элемента внутри другого, да ещё и по индексу. Во втором случае потенциальных проблем меньше, но не всегда на нужной кнопке будет текст, там вообще может быть картинка. Поэтому давайте познакомимся с универсальным способом поиска через **запросы XPATH**.

### Часть 2:  запросы XPATH

Немного теории.

**XML** (от *eXtended MarkUp Language*) – язык разметки, только в отличие от HTML не позволяет регулировать внешний вид страницы, а просто хранит данные в виде строки с удобными тэгами.

На XML-файл можно смотреть как на хранилище, откуда по запросу динамически подгружаются данные для подстановки в HTML-файл. Смысл: когда нам нужно постоянно обновлять информацию на веб-странице (каталоги товаров в магазине, данные о погоде, курсе валют), не нужно каждый раз переписывать HTML-файл, достаточно изменить XML-файл, а из него уже информация «подтянется» на страницу с помощью запроса, написанного на JavaScript.

**XPATH** (от *XML Path Language*) – язык запросов в XML-файлу, который можно использовать и для HTML тоже.

Примеры запросов (честно взяты [отсюда](https://www.w3schools.com/xml/xpath_intro.asp), очень полезный тьюториал по XPATH, у них же есть классные материалы по XML):

* `//title[@lang]`: все элементы с тэгом `<title>`, имеющие атрибут `lang`;
* `//title[@lang='en']` : все элементы с тэгом `<title>`, имеющие атрибут `lang`, равный `'en'`;
* `//title[@*]`: все элементы с тэгом `<title>`, имеющие хоть какие-нибудь атрибуты.


Давайте воспользуемся поиском с помощью запроса XPATH и найдём стрелку-ссылку на последнюю страницу (внимание на атрибут `aria-label`).

In [24]:
# поиск по атрибуту aria-label
# проблема – элемент с Next не один

br.find_elements(By.XPATH, "//a[@aria-label='Next']")

[<selenium.webdriver.remote.webelement.WebElement (session="4f308cb63f873f9d0ce50893811fd93a", element="B17C842BC9A8E4E3FF0E62DB02EC7F69_element_27")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4f308cb63f873f9d0ce50893811fd93a", element="B17C842BC9A8E4E3FF0E62DB02EC7F69_element_28")>]

In [25]:
# более надежный способ – найти через текст »»
# ищем тэг span внутри тэга a (отсюда .// – точка отвечает за вложенность),
# такой, что текст внутри равен »»

a = br.find_element(By.XPATH, "//a[.//span[text()='»»']]")

In [26]:
# вот ссылка на последнюю страницу – забираем
# атрибут href
# а в самой ссылке есть номер последней страницы
# будет понятно, сколько раз запускать цикл for для пролистывания

a.get_attribute("href")

'https://www.biblio-globus.ru/catalog/search?query=Python&page=10&sort=5&instock=on&cat=0&isdiscount='

Собственно, теперь можно через простое форматирование строк создать ссылки на страницы с результатами от 1 до 10, написать функцию для выгрузки результатов с одной страницы, используя наш код для обработки карточек товаров выше, и запустить цикл for для извлечения всех-всех результатов.

In [ ]:
# нужна f-строка – вместо page=10 подставляем числа от 1 до 10
# https://www.biblio-globus.ru/catalog/search?query=Python&page=10&sort=5&instock=on&cat=0&isdiscount=

In [27]:
from time import sleep

In [28]:
all_results = []

for i in range(1, 11):
    href = f"https://www.biblio-globus.ru/catalog/search?query=Python&page={i}&sort=5&instock=on&cat=0&isdiscount="
    
    # переходим на страницу, забираем код
    br.get(href)
    html = br.page_source
    soup = BeautifulSoup(html)
    
    # ищем карточки с результатами
    # применяем функцию get_item()
    # записываем список «чистых» результатов в all_results
    # ждем 2 секунды
    divs = soup.find_all("div", {"class" : "product"})
    res = [get_item(d) for d in divs]
    all_results.extend(res)
    sleep(2)

In [43]:
import pandas as pd

In [44]:
fin = pd.DataFrame(all_results)
fin.columns = ["name", "link", "price"]

In [45]:
fin.head()

,name,link,price
0,Папка на резинках пластиковая ErichKrause® Pyt...,https://www.biblio-globus.ru//product/10730131,159 ₽
1,Папка на 4 кольцах пластиковая ErichKrause® Tr...,https://www.biblio-globus.ru//product/10726752,269 ₽
2,Python глазами хакера,https://www.biblio-globus.ru//product/10823441,509 ₽ 458 ₽
3,"Справочник PYTHON. Кратко, быстро, под рукой",https://www.biblio-globus.ru//product/10776656,569 ₽ 512 ₽
4,"Python на примерах. Практика, практика и тольк...",https://www.biblio-globus.ru//product/10914448,579 ₽ 521 ₽


In [46]:
# разбиваем цену на части, где возможно
prices = fin["price"].str.split(" ₽", expand = True)
prices.head()

,0,1,2
0,159,,None
1,269,,None
2,509,458,
3,569,512,
4,579,521,


In [47]:
# добавляем столбцы в fin
fin["price"] = prices[0]
fin["price_discount"] = prices[1]

In [49]:
# переводим цены в целочисленный формат

fin["price"] = fin["price"].apply(lambda x: None if x == " " else int(x))
fin["price_discount"] = fin["price_discount"].apply(lambda x: None if x == "" else int(x))

In [50]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            116 non-null    object 
 1   link            116 non-null    object 
 2   price           116 non-null    int64  
 3   price_discount  56 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.8+ KB


In [51]:
fin.head()

,name,link,price,price_discount
0,Папка на резинках пластиковая ErichKrause® Pyt...,https://www.biblio-globus.ru//product/10730131,159,NaN
1,Папка на 4 кольцах пластиковая ErichKrause® Tr...,https://www.biblio-globus.ru//product/10726752,269,NaN
2,Python глазами хакера,https://www.biblio-globus.ru//product/10823441,509,458.0
3,"Справочник PYTHON. Кратко, быстро, под рукой",https://www.biblio-globus.ru//product/10776656,569,512.0
4,"Python на примерах. Практика, практика и тольк...",https://www.biblio-globus.ru//product/10914448,579,521.0
